In [1]:
import requests
import http.client
import json, codecs
import numpy as np
import re 
import datetime
import pandas as pd
import urllib
import time
import pickle
from concurrent.futures import ThreadPoolExecutor

In [2]:
def get_vacancy_data(vac_id):
    url = "http://api.hh.ru/vacancies/{}".format(vac_id)
    headers = {"User-Agent": "hh-recommender"}
    sess = requests.session()
    r = sess.post(url, headers=headers)
    json_data = json.loads(r.text)
    time.sleep(0.05)
    return json_data

In [3]:
def cleaning_data(data, useless_key):
    if data['description'] == 'Not Found':
        return None
    else:
        for key in useless_key:
            try:
                del data[key]
            except KeyError:
                pass
        return data

In [4]:
def save_data(data, id_range):
    with open(('hh_parsing_{}.json'.format(id_range[-1])), 'ab') as f:
        json.dump(data, codecs.getwriter('utf-8')(f), ensure_ascii=False)
        f.close()

In [23]:
def hh_pars(id_number, id_range, useless_key):    
    try:
        data = get_vacancy_data(id_number)
    except Exception:
        data = {'description': "Not Found"}
        with open('/home/andrew/HH_Project/Error_Vac.pickle', 'ab') as file:
            pickle.dump(id_number, file, protocol=pickle.HIGHEST_PROTOCOL)
            file.close()
            time.sleep(0.5)
            pass
    clean_data = cleaning_data(data, useless_key)   
    save_data(clean_data, id_range)

In [26]:
useless_key = ['allow_messages', 'branded_description', 'apply_alternate_url', 'code', 'department', 
               'response_letter_required', 'test', 'premium', 'negotiations_url', 'quick_responses_allowed', 
               'relations', 'suitable_resumes_url']
vac_id = range(500000, 1000000)

In [ ]:
for i in vac_id:
    hh_pars(i, vac_id, useless_key)